In [ ]:
# analysis: how does this user's swaps make money
import math
import pandas as pd

from utils import utils

TICK_BASE = 1.0001

def tick_to_price(tick):
    return TICK_BASE ** tick * 10 ** 12

def price_to_tick(price):
    return math.log(price/(10**12), TICK_BASE)


# new csv resolved USDT related swaps => converted to USDC/WETH trading pair
df = pd.read_csv("../data/user_arb_swap_new.csv")
df["datetime"] = df["timestamp"].apply(utils.utc_timestamp_to_date)
df["tick_price"] = df["tick"].apply(tick_to_price)
df.head()

In [ ]:
user_data = df.to_dict("records")

for item in user_data:
    if item["tokenIn"] == "USDC":
        item["trade_type"] = "BUY"
        item["trade_eth"] = int(item["amountOut"]) / (10**18)
        item["trade_price"] = item["amountOutUSD"] / item["trade_eth"]
    if item["tokenIn"] == "WETH":
        item["trade_type"] = "SELL"
        item["trade_eth"] = int(item["amountIn"]) / (10**18)
        item["trade_price"] = item["amountInUSD"] / item["trade_eth"]
    item["price_diff"] = item["trade_price"] - item["tick_price"]

    trade_tick = price_to_tick(item["trade_price"])
    item["price_diff_in_tick"] = round(trade_tick - item["tick"])

user_df = pd.DataFrame(user_data)
user_df.head()

In [ ]:
init_price = 1471.206304
init_eth = 100
init_usd = 100 * init_price

# calculate eth, usd in every step to know least investment, considering about trading fee
user_data = user_df.to_dict("records")

last_price = init_price
last_eth = init_eth
last_usd = init_usd

current_usd = 0
current_eth = 0
for item in user_data:
    item["uniswap_fees"] = item["amountInUSD"] * 0.0005
    item["fees"] = item["amountInUSD"] - item["amountOutUSD"]
    if item["tokenIn"] == "USDC":
        current_usd = last_usd - item["amountInUSD"]
        current_eth = last_eth + int(item["amountOut"]) / (10 ** 18)
    if item["tokenIn"] == "WETH":
        current_usd = last_usd + item["amountOutUSD"]
        current_eth = last_eth - int(item["amountIn"]) / (10 ** 18)

    item["current_usd"] = current_usd
    item["current_eth"] = current_eth

    item["tick_portfolio_profit_usd"] = current_usd + current_eth * item["tick_price"] - init_usd - init_eth * item["tick_price"]
    item["trade_portfolio_profit_usd"] = current_usd + current_eth * item["trade_price"] - init_usd - init_eth * item["trade_price"]
    # profit = item["price"] * current_eth + current_usd - item["price"] * init_eth - init_usd
    # item["profit"] = profit

    last_usd = current_usd
    last_eth = current_eth


user_df = pd.DataFrame(user_data)
user_df.head()

In [ ]:
columns = ["datetime", "trade_type", "trade_eth", "trade_price", "current_usd", "current_eth", "trade_portfolio_profit_usd", "tick_portfolio_profit_usd", "tick_price", "price_diff", "price_diff_in_tick", "fees", "uniswap_fees", "tick", "amountIn", "amountOut", "amountInUSD", "amountOutUSD"]
user_df[columns].head(20)

In [ ]:
current_price = 1468.75581
print(176209.200205 + 80.194769 * current_price - 100 * 1471.206304 - 100 * current_price)

In [ ]:
current_price = 1466.06601
print(185166.445632 + 74.083996 * current_price - 100 * 1471.206304 - 100 * current_price)

In [ ]:
data = user_df[["trade_type", "trade_eth", "trade_price", "fees"]].to_dict("records")[:8]
len(data)

In [ ]:
sell_usd = 0
sell_amount = 0
for i, item in enumerate(data):
    if item["trade_type"] == "SELL":
        sell_usd += item["trade_eth"] * item["trade_price"]
        sell_amount += item["trade_eth"]
        print(i+1, sell_usd/sell_amount)

In [ ]:
# q1: 为什么以 trade_price 计价，fees 与 trade_portfolio_profit_usd 相等